# GTC 2017 Numba Tutorial Notebook 1: Numba Basics

## What is Numba?

Numba is a **just-in-time**, **type-specializing**, **function compiler** for accelerating **numerically-focused** Python.  That's a long list, so let's break down those terms:

 * **function compiler**: Numba compiles Python functions, not entire applications, and not parts of functions.  Numba does not replace your Python interpreter, but is just another Python module that can turn a function into a (usually) faster function. 
 * **type-specializing**: Numba speeds up your function by generating a specialized implementation for the specific data types you are using.  Python functions are designed to operate on generic data types, which makes them very flexible, but also very slow.  In practice, you only will call a function with a small number of argument types, so Numba will generate a fast implementation for each set of types.
 * **just-in-time**: Numba translates functions when they are first called.  This ensures the compiler knows what argument types you will be using.  This also allows Numba to be used interactively in a Jupyter notebook just as easily as a traditional application
 * **numerically-focused**: Currently, Numba is focused on numerical data types, like `int`, `float`, and `complex`.  There is very limited string processing support, and many string use cases are not going to work well on the GPU.  To get best results with Numba, you will likely be using NumPy arrays.

## Requirements

Numba supports a wide range of operating systems:

 * Windows 7 and later, 32 and 64-bit
 * macOS 10.9 and later, 64-bit
 * Linux (most anything >= RHEL 5), 32-bit and 64-bit

and Python versions:

 * Python 2.7, 3.3-3.6
 * NumPy 1.8 and later

and a very wide range of hardware:

* x86, x86_64/AMD64 CPUs
* NVIDIA CUDA GPUs (Compute capability 3.0 and later, CUDA 7.5 and later)
* AMD GPUs (experimental patches)
* ARM (experimental patches)

For this tutorial, we will be using Linux 64-bit and CUDA 8.

## First Steps

Let's write our first Numba function and compile it for the **CPU**.  The Numba compiler is typically enabled by applying a *decorator* to a Python function.  Decorators are functions that transform Python functions.  Here we will use the CPU compilation decorator:

In [1]:
from numba import jit
import math

@jit
def hypot(x, y):
    # Implementation from https://en.wikipedia.org/wiki/Hypot
    x = abs(x);
    y = abs(y);
    t = min(x, y);
    x = max(x, y);
    t = t / x;
    return x * math.sqrt(1+t*t)

The above code is equivalent to writing:
``` python
def hypot(x, y):
    x = abs(x);
    y = abs(y);
    t = min(x, y);
    x = max(x, y);
    t = t / x;
    return x * math.sqrt(1+t*t)
    
hypot = jit(hypot)
```
This means that the Numba compiler is just a function you can call whenever you want!

Let's try out our hypotenuse calculation:

In [2]:
hypot(3.0, 4.0)

5.0

The first time we call `hypot`, the compiler is triggered and compiles a machine code implementation for float inputs.  Numba also saves the original Python implementation of the function in the `.py_func` attribute, so we can call the original Python code to make sure we get the same answer:

In [3]:
hypot.py_func(3.0, 4.0)

5.0

## Benchmarking

An important part of using Numba is measuring the performance of your new code.  Let's see if we actually sped anything up.  The easiest way to do this in the Jupyter notebook is to use the `%timeit` magic function.  Let's first measure the speed of the original Python:

In [4]:
%timeit hypot.py_func(3.0, 4.0)

743 ns ± 6.48 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


The `%timeit` magic runs the statement many times to get an accurate estimate of the run time.  It also returns the best time by default, which is useful to reduce the probability that random background events affect your measurement.  The best of 3 approach also ensures that the compilation time on the first call doesn't skew the results:

In [5]:
%timeit hypot(3.0, 4.0)

147 ns ± 0.615 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Numba did a pretty good job with this function.  It's more than 4x faster than the pure Python version.

Of course, the `hypot` function is already present in the Python module:

In [6]:
%timeit math.hypot(3.0, 4.0)

147 ns ± 0.321 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


Python's built-in is even faster than Numba!  This is because Numba does introduce some overhead to each function call that is larger than the function call overhead of Python itself.  Extremely fast functions (like the above one) will be hurt by this.

(However, if you call one Numba function from another one, there is very little function overhead, sometimes even zero if the compiler inlines the function into the other one.)

## How does Numba work?

The first time we called our Numba-wrapped `hypot` function, the following process was initiated:

![Numba Flowchart](img/numba_flowchart.png "The compilation process")

We can see the result of type inference by using the `.inspect_types()` method, which prints an annotated version of the source code:

In [7]:
hypot.inspect_types()

hypot (float64, float64)
--------------------------------------------------------------------------------
# File: <ipython-input-1-2d77e48c1ef7>
# --- LINE 4 --- 

@jit

# --- LINE 5 --- 

def hypot(x, y):

    # --- LINE 6 --- 

    # Implementation from https://en.wikipedia.org/wiki/Hypot

    # --- LINE 7 --- 
    # label 0
    #   x = arg(0, name=x)  :: float64
    #   y = arg(1, name=y)  :: float64
    #   $2load_global.0 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
    #   $6call_function.2 = call $2load_global.0(x, func=$2load_global.0, args=[Var(x, <ipython-input-1-2d77e48c1ef7>:7)], kws=(), vararg=None)  :: (float64,) -> float64
    #   del x
    #   del $2load_global.0
    #   x.1 = $6call_function.2  :: float64
    #   del $6call_function.2

    x = abs(x);

    # --- LINE 8 --- 
    #   $10load_global.3 = global(abs: <built-in function abs>)  :: Function(<built-in function abs>)
    #   $14call_function.5 = call $10load_global.3(y, func=$10lo

Note that Numba's type names tend to mirror the NumPy type names, so a Python `float` is a `float64` (also called "double precision" in other languages).  Taking a look at the data types can sometimes be important in GPU code because the performance of `float32` and `float64` computations will be very different on CUDA devices.  An accidental upcast can dramatically slow down a function.

## When Things Go Wrong

Numba cannot compile all Python code.  Some functions don't have a Numba-translation, and some kinds of Python types can't be efficiently compiled at all (yet).  For example, Numba does not support dictionaries (as of this tutorial):

In [8]:
@jit
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='value'))

<ipython-input-8-9ab63214e97a>:1: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "cannot_compile" failed type inference due to: non-precise type pyobject
[1] During: typing of argument at <ipython-input-8-9ab63214e97a> (3)

File "<ipython-input-8-9ab63214e97a>", line 3:
def cannot_compile(x):
    return x['key']
    ^

  @jit
C:\Users\corsair\Anaconda3\lib\site-packages\numba\core\object_mode_passes.py:178: NumbaWarning: Function "cannot_compile" was compiled in object mode without forceobj=True.

File "<ipython-input-8-9ab63214e97a>", line 2:
@jit
def cannot_compile(x):
^

  state.func_ir.loc))
C:\Users\corsair\Anaconda3\lib\site-packages\numba\core\object_mode_passes.py:188: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit http://numba.pydata.org/numba-doc/latest/reference/deprecation.html#deprecati

'value'

Wait, what happened??  By default, Numba will fall back to a mode, called "object mode," which does not do type-specialization.  Object mode exists to enable other Numba functionality, but in many cases, you want Numba to tell you if type inference fails.  You can force "nopython mode" (the other compilation mode) by passing arguments to the decorator:

In [9]:
@jit(nopython=True)
def cannot_compile(x):
    return x['key']

cannot_compile(dict(key='value'))

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1m[1mnon-precise type pyobject[0m
[0m[1m[1] During: typing of argument at <ipython-input-9-42c374763781> (3)[0m
[1m
File "<ipython-input-9-42c374763781>", line 3:[0m
[1mdef cannot_compile(x):
[1m    return x['key']
[0m    [1m^[0m[0m

This error may have been caused by the following argument(s):
- argument 0: [1mcannot determine Numba type of <class 'dict'>[0m


Now we get an exception when Numba tries to compile the function, with an error that says:
```
- argument 0: cannot determine Numba type of <class 'dict'>
```
which is the underlying problem.

We will see other `@jit` decorator arguments in future sections.

# Exercise

Below is a function that loops over two input NumPy arrays and puts their sum into the output array.  Modify this function to call the `hypot` function we defined above.  We will learn a more efficient way to write such functions in a future section.

(Make sure to execute all the cells in this notebook so that `hypot` is defined.)

In [13]:
@jit(nopython=True)
def ex1(x, y, out):
    for i in range(x.shape[0]):
        out[i] = hypot(x[i], y[i])

In [14]:
import numpy as np

in1 = np.arange(10, dtype=np.float64)
in2 = 2 * in1 + 1
out = np.empty_like(in1)

print('in1:', in1)
print('in2:', in2)

ex1(in1, in2, out)

print('out:', out)

in1: [0. 1. 2. 3. 4. 5. 6. 7. 8. 9.]
in2: [ 1.  3.  5.  7.  9. 11. 13. 15. 17. 19.]
out: [ 1.          3.16227766  5.38516481  7.61577311  9.8488578  12.08304597
 14.31782106 16.55294536 18.78829423 21.02379604]


In [15]:
# This test will fail until you fix the ex1 function
np.testing.assert_almost_equal(out, np.hypot(in1, in2))